In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import gseapy as gp
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand

p = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))))
sys.path.append(p)

from scripts.utils import *
from scripts.plotting import *
from scripts.mane_utils import *

In [2]:
# def parse_blastp(blastp_file, orf_file, ofile):
    
#     # get all the hits
#     df = pd.read_csv(blastp_file, sep='\t',header=None,
#                      names=['qseqid', 'sseqid', 'pident',
#                             'length', 'mismatch', 'gapopen',
#                             'qstart', 'qend', 'sstart', 'send', 
#                             'evalue', 'bitscore', 'slen', 'qlen'])
    
    
#     pdb.set_trace()

#     q_name_split = df.qseqid.str.split(':', expand=True)
#     df['q_frame'] = q_name_split[4]
#     df['q_rel_start'] = q_name_split[7]
#     df['q_rel_end'] = q_name_split[8]
#     df['q_nuc_start'] = q_name_split[5]
#     df['q_nuc_end'] = q_name_split[6]
#     df['tid'] = df.qseqid.str.split(';',expand=True)[1]

#     # percentage of subject that was aligned. for instance, if 
#     # this is 100, then the query sequence matched 100% of
#     # a protein sequence
#     df['s_align_len'] = df['send'] + 1 - df['sstart']
#     df['s_align_percent'] = (df.s_align_len/df.slen)*100

#     # match flags
#     df['match_flag'] = 'none'
#     df.loc[df.qseqid.str.contains('missing_nucleotides'), 'match_flag'] = 'missing_nucleotides'
#     df.loc[df.s_align_percent >= 50, 'match_flag'] = '50_match'
#     df.loc[df.s_align_percent >= 90, 'match_flag'] = '90_match'
#     df.loc[df.s_align_percent == 100, 'match_flag'] = 'full_match'

#     # reformat stuff
#     cols = ['tid', 'q_frame', 
#             'q_nuc_start', 
#             'q_nuc_end', 
#             'q_rel_start', 
#             'q_rel_end',
#             's_name',
#             'match_flag', 
#             's_align_percent',
#             'ident_percent', 
#             'q_name']

#     # what is the difference between s_align_percent and ident_percent?
#     # idc because it's not used in the heuristic.
#     # but s_align_percent is and it uses s_len, which I don't have

#     df.rename({'qseqid':'q_name',
#                'sseqid': 's_name',
#                'pident': 'ident_percent'}, 
#                axis=1, inplace=True)

#     df = df[cols]
#     df['full_orf'] = df.q_rel_start != '1'

#     # pick one orf for each transcript
#     # - first pick highest alignment % with subject (ie, is this a complete protein?)
#     # - then, pick complete orfs
#     # - finally, pick longest orf
#     # df = df.sort_values(by=['s_align_percent', 'full_orf', 'qlen'],
#     #                     ascending=[False, False, False])
#     return df 
#     df = df.sort_values(by=['s_align_percent', 'full_orf'],
#                         ascending=[False, False])
#     df = df.drop_duplicates(subset=['tid'], keep='first')

#     df.drop('full_orf', axis=1, inplace=True)
    
#     # get all the orfs for the non-hit tids
#     tids = df.tid.tolist()
#     no_hit_df = get_no_hit_blastp_entries(orf_file, tids)
    
#     # concat
#     df = pd.concat([df, no_hit_df])
    
#     # save to output
#     df.to_csv(ofile, sep='\t', header=None)
    
# def get_no_hit_blastp_entries(fname, tids):
#     """
#     Parameters:
#         fname (str): Path to orf_fa
#         tids (list of str): List of transcript ids that hits were found for
#     """
#     # go back through tids that didn't get a blast hit and give them the longest orf
#     df = read_orf_fa(fname)

#     # limit only to missing tids
#     df = df.loc[~df.tid.isin(tids)]

#     cols = ['tid',
#             'q_frame', 
#             'q_nuc_start', 
#             'q_nuc_end', 
#             'q_rel_start', 
#             'q_rel_end',
#             's_name',
#             'match_flag', 
#             's_align_percent',
#             'ident_percent', 
#             'q_name']

#     # default to missing nucleotides settings
#     df['q_frame'] = 'no_frame'
#     df['q_nuc_start'] = -1
#     df['q_nuc_end'] = -1
#     df['q_rel_start'] = -1
#     df['q_rel_end'] = -1
#     df['s_name'] = 'none'
#     df['match_flag'] = 'missing_nucleotides'
#     df['s_align_percent'] = -1
#     df['ident_percent'] = -1
#     df['q_name'] = df.id

#     # but for those that aren't, pull out orf details
#     inds = df.loc[~df.id.str.contains('missing_nucleotides')].index.tolist()
#     q_name_split = df.loc[inds].id.str.split(':', expand=True)
#     df.loc[inds, 'q_frame'] = q_name_split[4]
#     df.loc[inds, 'q_nuc_start'] = q_name_split[5].astype(int)
#     df.loc[inds, 'q_nuc_end'] = q_name_split[6].astype(int)
#     df.loc[inds, 'q_rel_start'] = q_name_split[7].astype(int)
#     df.loc[inds, 'q_rel_end'] = q_name_split[8].astype(int)
#     df.loc[inds, 'match_flag'] = 'no_hit'
#     df.loc[inds, 's_align_percent'] = 0
#     df.loc[inds, 'ident_percent'] = 0

#     df['full_orf'] = df.q_rel_start != 1
    
#     # pick one orf for each transcript
#     # - pick longest complete orf
#     df = df.sort_values(by=['full_orf', 'len'],
#                         ascending=[False, False])
#     df = df.drop_duplicates(subset=['tid'], keep='first')
    
#     df.drop(['full_orf', 'id', 'seq', 'len'], axis=1, inplace=True)
    
#     return df

In [3]:
blastp_file = 'human_blastp.6.out'
orf_file = 'human_orf.fa'
ofile = 'test_blastp_parsed.tsv'
df = parse_blastp(blastp_file, orf_file, ofile)

In [5]:
df = pd.read_csv(ofile, sep='\t', header=None)
# df.tail()
# df.head()

,0,1,2,3,4,5,6,7,8,9,10
231117,"ENSG00000207554.1;ENSG00000207554[1,1,1];NA;NA",F2,1,12,1,3,none,no_hit,0.0,0.0,"ENSG00000207554.1;ENSG00000207554[1,1,1];NA;NA..."
231118,"ENSG00000276081.1;ENSG00000276081[1,1,1];NA;NA",F3,2,10,1,2,none,no_hit,0.0,0.0,"ENSG00000276081.1;ENSG00000276081[1,1,1];NA;NA..."
231119,"ENSG00000221445.3;ENSG00000221445[1,1,1];NA;NA",F1,0,8,1,2,none,no_hit,0.0,0.0,"ENSG00000221445.3;ENSG00000221445[1,1,1];NA;NA..."
231120,"ENSG00000284129.1;ENSG00000284129[1,1,1];NA;NA",F2,1,6,1,1,none,no_hit,0.0,0.0,"ENSG00000284129.1;ENSG00000284129[1,1,1];NA;NA..."
231121,"ENSG00000202515.1;ENSG00000202515[1,1,1];NA;NA",F1,0,5,1,1,none,no_hit,0.0,0.0,"ENSG00000202515.1;ENSG00000202515[1,1,1];NA;NA..."


In [9]:
# what's STILL going on w/ eln
gid = 'ENSG00000049540'
tid = 'ENSG00000049540[1,1,1]'
df = df.loc[df.tid==tid]

In [12]:
df.loc[df.full_orf==True].sort_values(by=['s_align_percent', 'full_orf'],
                        ascending=[False, False])

,tid,q_frame,q_nuc_start,q_nuc_end,q_rel_start,q_rel_end,s_name,match_flag,s_align_percent,ident_percent,q_name,full_orf
3987345,"ENSG00000049540[1,1,1]",F3,137,2311,46,769,ENSP00000510104.1,full_match,100.000000,100.000,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...",True
3987346,"ENSG00000049540[1,1,1]",F3,137,2311,46,769,ENSP00000313565.6,full_match,100.000000,100.000,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...",True
3987347,"ENSG00000049540[1,1,1]",F3,137,2311,46,769,ENSP00000399499.1,full_match,100.000000,100.000,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...",True
3987348,"ENSG00000049540[1,1,1]",F3,137,2311,46,769,ENSP00000369950.5,full_match,100.000000,100.000,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...",True
3987349,"ENSG00000049540[1,1,1]",F3,137,2311,46,769,ENSP00000369926.4,full_match,100.000000,100.000,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...",True
3987350,"ENSG00000049540[1,1,1]",F3,137,2311,46,769,ENSP00000403961.1,full_match,100.000000,100.000,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...",True
3987351,"ENSG00000049540[1,1,1]",F3,137,2311,46,769,ENSP00000252034.7,full_match,100.000000,100.000,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...",True
3987352,"ENSG00000049540[1,1,1]",F3,137,2311,46,769,ENSP00000391129.1,full_match,100.000000,100.000,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...",True
3987353,"ENSG00000049540[1,1,1]",F3,137,2311,46,769,ENSP00000411092.1,full_match,100.000000,100.000,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...",True
3987354,"ENSG00000049540[1,1,1]",F3,137,2311,46,769,ENSP00000349540.5,full_match,100.000000,100.000,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...",True


In [5]:
df = pd.read_csv(ofile, sep='\t', header=None, names=['tid',
            'q_frame', 
            'q_nuc_start', 
            'q_nuc_end', 
            'q_rel_start', 
            'q_rel_end',
            's_name',
            'match_flag', 
            's_align_percent',
            'ident_percent', 
            'q_name'])
# df = pd.read_csv(ofile, sep='\t', header=None)
# df.tail()

In [6]:
df.head()

,tid,q_frame,q_nuc_start,q_nuc_end,q_rel_start,q_rel_end,s_name,match_flag,s_align_percent,ident_percent,q_name
0,"ENSG00000000460[3,10,3]",F1,423,3158,142,1052,ENSP00000286031.6,full_match,100.0,100.0,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;..."
16,"ENSG00000000460[1,10,3]",F3,113,2848,38,948,ENSP00000286031.6,full_match,100.0,100.0,"ENSG00000000460.16;ENSG00000000460[1,10,3];NA;..."
32,"ENSG00000000460[1,10,6]",F3,113,2848,38,948,ENSP00000286031.6,full_match,100.0,100.0,"ENSG00000000460.16;ENSG00000000460[1,10,6];NA;..."
48,"ENSG00000000460[1,10,1]",F3,113,2848,38,948,ENSP00000286031.6,full_match,100.0,100.0,"ENSG00000000460.16;ENSG00000000460[1,10,1];NA;..."
56,"ENSG00000000460[3,11,3]",F2,595,3069,199,1022,ENSP00000389257.3,full_match,100.0,100.0,"ENSG00000000460.16;ENSG00000000460[3,11,3];NA;..."


In [5]:
df = pd.read_csv('../../human/protein_pred/smol_blastp.6.out', sep='\t',header=None,
                 names=['qseqid', 'sseqid', 'pident',
                        'length', 'mismatch', 'gapopen',
                        'qstart', 'qend', 'sstart', 'send', 
                        'evalue', 'bitscore', 'slen', 'qlen'])

q_name_split = df.qseqid.str.split(':', expand=True)
df['q_frame'] = q_name_split[4]
df['q_rel_start'] = q_name_split[7]
df['q_rel_end'] = q_name_split[8]
df['q_nuc_start'] = q_name_split[5]
df['q_nuc_end'] = q_name_split[6]
df['tid'] = df.qseqid.str.split(';',expand=True)[1]

# percentage of subject that was aligned. for instance, if 
# this is 100, then the query sequence matched 100% of
# a protein sequence
df['s_align_len'] = df['send'] + 1 - df['sstart']
df['s_align_percent'] = (df.s_align_len/df.slen)*100

# match flags
df['match_flag'] = 'none'
df.loc[df.qseqid.str.contains('missing_nucleotides'), 'match_flag'] = 'missing_nucleotides'
df.loc[df.s_align_percent >= 50, 'match_flag'] = '50_match'
df.loc[df.s_align_percent >= 90, 'match_flag'] = '90_match'
df.loc[df.s_align_percent == 100, 'match_flag'] = 'full_match'

# reformat stuff
cols = ['tid', 'q_frame', 
        'q_nuc_start', 
        'q_nuc_end', 
        'q_rel_start', 
        'q_rel_end',
        's_name',
        'match_flag', 
        's_align_percent',
        'ident_percent', 
        'q_name']

# what is the difference between s_align_percent and ident_percent?
# idc because it's not used in the heuristic.
# but s_align_percent is and it uses s_len, which I don't have

df.rename({'qseqid':'q_name',
           'sseqid': 's_name',
           'pident': 'ident_percent'}, 
           axis=1, inplace=True)

df = df[cols]
df['full_orf'] = df.q_rel_start != '1'

# pick one orf for each transcript
# - first pick highest alignment % with subject (ie, is this a complete protein?)
# - then, pick complete orfs
# - finally, pick longest orf
# df = df.sort_values(by=['s_align_percent', 'full_orf', 'qlen'],
#                     ascending=[False, False, False])
df = df.sort_values(by=['s_align_percent', 'full_orf'],
                    ascending=[False, False])
df = df.drop_duplicates(subset=['tid'], keep='first')

df.drop('full_orf', axis=1, inplace=True)

In [139]:
# df.loc[df.tid=='ENSG00000000460[1,10,3]']

In [140]:
# temp.loc[temp.tid=='ENSG00000000460[1,10,3]']

In [141]:
# df.loc[df.full_orf==False]

In [142]:
# input
tids = df.tid.tolist()
fname = 'human_orf.fa'

In [174]:
df.loc[df.match_flag=='missing_nucleotides']
# df.loc[(df.duplicated(subset='tid', keep=False))&(df.match_flag=='missing_nucleotides')]
# df.head()
# df.loc[df.tid=='ENSG00000151151[1,3,1]']

,id,tid,seq,len,q_frame,q_nuc_start,q_nuc_end,q_rel_start,q_rel_end,s_name,match_flag,s_align_percent,ident_percent,q_name,full_orf


,id,tid,seq,len,q_frame,q_nuc_start,q_nuc_end,q_rel_start,q_rel_end,s_name,match_flag,s_align_percent,ident_percent,q_name,full_orf
67,"ENSG00000000971.15;ENSG00000000971[1,41,15];NA...","ENSG00000000971[1,41,15]",MDLLYLRRLFIRRMNDFNINVTWVMNTVKEEMLYALNLDGVRCLHV...,63,F2,1063,1254,355,417,none,no_hit,0,0,"ENSG00000000971.15;ENSG00000000971[1,41,15];NA...",True


In [96]:
df.q_name.values[0]
df.head()

,tid,q_frame,q_nuc_start,q_nuc_end,q_rel_start,q_rel_end,s_name,match_flag,s_align_percent,ident_percent,q_name
0,"ENSG00000000460[3,10,3]",F1,423,3158,142,1052,ENSP00000286031.6,full_match,100.0,100.0,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;..."
1,"ENSG00000000460[3,10,3]",F1,423,3158,142,1052,ENSP00000352276.4,full_match,100.0,100.0,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;..."
2,"ENSG00000000460[3,10,3]",F1,423,3158,142,1052,ENSP00000490194.1,full_match,100.0,100.0,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;..."
3,"ENSG00000000460[3,10,3]",F1,423,3158,142,1052,ENSP00000490333.1,full_match,100.0,100.0,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;..."
4,"ENSG00000000460[3,10,3]",F1,423,3158,142,1052,ENSP00000389257.3,full_match,100.0,100.0,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;..."


In [56]:
temp = pd.read_csv('human_blastp.tsv', sep='\t', header=None,
                  names=['big_tid', 'frame', 'start', 'stop', 'rel_start', 'rel_stop', 
                        's_name', 'match_flag', 's_align_percent', 'ident_percent', 'q_name'])

In [97]:
temp.loc[temp.q_name=='ENSG00000000460.16;ENSG00000000460[3,10,3];NA;NA::chr1:169794679-169853135(+):F1:423:3158:142:1052:911:M']

,big_tid,frame,start,stop,rel_start,rel_stop,s_name,match_flag,s_align_percent,ident_percent,q_name
0,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;NA",F1,423,3158,142,1052,ENSP00000286031.6,full_match,100,100,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;..."


In [78]:
temp.loc[temp['s_align_percent'] != temp['ident_percent']].s_name.values[0]


'ENSP00000497116.1'

In [79]:
temp.loc[temp['s_align_percent'] != temp['ident_percent']].q_name.values[0]



'ENSG00000000971.15;ENSG00000000971[3,37,1];NA;NA::chr1:196676937-196747666(+):F2:6148:6843:2050:2280:231:M'

In [80]:
temp.loc[temp['s_align_percent'] != temp['ident_percent']].head(1)

,big_tid,frame,start,stop,rel_start,rel_stop,s_name,match_flag,s_align_percent,ident_percent,q_name
21,"ENSG00000000971.15;ENSG00000000971[3,37,1];NA;NA",F2,6148,6843,2050,2280,ENSP00000497116.1,full_match,100,43,"ENSG00000000971.15;ENSG00000000971[3,37,1];NA;..."


In [69]:
temp.loc[temp.q_name=='ENSG00000151150.21;ENSG00000151150[2,3,4];NA;NA::chr10:60138466-60140565(-):F1:198:416:67:138:72:M']

,big_tid,frame,start,stop,rel_start,rel_stop,s_name,match_flag,s_align_percent,ident_percent,q_name
29367,"ENSG00000151150.21;ENSG00000151150[2,3,4];NA;NA",F1,198,416,67,138,ENSP00000483483.1,full_match,100,100,"ENSG00000151150.21;ENSG00000151150[2,3,4];NA;N..."


In [59]:
temp.loc[temp.q_name.str.contains('missing_nucleotides')].q_name.values[0]

'ENSG00000151151.5;ENSG00000151151[1,3,1];NA;NA::chr10:58191466-58267944(-):missing_nucleotides'

In [62]:
temp.loc[temp.match_flag=='no_hit']

,big_tid,frame,start,stop,rel_start,rel_stop,s_name,match_flag,s_align_percent,ident_percent,q_name
16,"ENSG00000000971.15;ENSG00000000971[1,41,15];NA;NA",F2,1063,1254,355,417,none,no_hit,0,0,"ENSG00000000971.15;ENSG00000000971[1,41,15];NA..."
17,"ENSG00000000971.15;ENSG00000000971[1,42,16];NA;NA",F2,1063,1254,355,417,none,no_hit,0,0,"ENSG00000000971.15;ENSG00000000971[1,42,16];NA..."
32,"ENSG00000001461.16;ENSG00000001461[1,6,5];NA;NA",F1,0,269,1,89,none,no_hit,0,0,"ENSG00000001461.16;ENSG00000001461[1,6,5];NA;N..."
33,"ENSG00000001461.16;ENSG00000001461[1,5,4];NA;NA",F1,0,269,1,89,none,no_hit,0,0,"ENSG00000001461.16;ENSG00000001461[1,5,4];NA;N..."
50,"ENSG00000004487.16;ENSG00000004487[11,18,1];NA;NA",F2,307,588,103,195,none,no_hit,0,0,"ENSG00000004487.16;ENSG00000004487[11,18,1];NA..."
...,...,...,...,...,...,...,...,...,...,...,...
231117,"ENSG00000252766.1;ENSG00000252766[1,1,1];NA;NA",F3,2,151,1,49,none,no_hit,0,0,"ENSG00000252766.1;ENSG00000252766[1,1,1];NA;NA..."
231118,"ENSG00000260197.1;ENSG00000260197[1,1,1];NA;NA",F2,640,771,214,256,none,no_hit,0,0,"ENSG00000260197.1;ENSG00000260197[1,1,1];NA;NA..."
231119,"ENSG00000277930.1;ENSG00000277930[1,1,1];NA;NA",F1,72,197,25,65,none,no_hit,0,0,"ENSG00000277930.1;ENSG00000277930[1,1,1];NA;NA..."
231120,"ENSG00000278212.1;ENSG00000278212[2,2,2];NA;NA",F1,147,341,50,113,none,no_hit,0,0,"ENSG00000278212.1;ENSG00000278212[2,2,2];NA;NA..."


In [37]:
df = pd.read_csv('test_blastp.tsv', sep='\t', header=None,
                 names=['big_tid', 'frame', 'start', 'stop', 'rel_start', 'rel_stop', 
                        's_name', 'match_flag', 's_align_percent', 'ident_percent', 'q_name'])

In [39]:
df.loc[df.q_name=='ENSG00000000460.16;ENSG00000000460[3,10,3];NA;NA::chr1:169794679-169853135(+):F1:423:3158:142:1052:911:M']

,big_tid,frame,start,stop,rel_start,rel_stop,s_name,match_flag,s_align_percent,ident_percent,q_name
0,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;NA",F1,423,3158,142,1052,ENSP00000286031.6,90_match,100,100,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;..."
1,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;NA",F1,423,3158,142,1052,ENSP00000352276.4,90_match,100,100,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;..."
2,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;NA",F1,423,3158,142,1052,ENSP00000490194.1,90_match,100,100,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;..."
3,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;NA",F1,423,3158,142,1052,ENSP00000490333.1,90_match,100,100,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;..."
4,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;NA",F1,423,3158,142,1052,ENSP00000389257.3,90_match,100,100,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;..."
5,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;NA",F1,423,3158,142,1052,ENSP00000490589.1,90_match,91,98,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;..."
6,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;NA",F1,423,3158,142,1052,ENSP00000490772.1,90_match,91,98,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;..."
7,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;NA",F1,423,3158,142,1052,ENSP00000489730.1,90_match,91,98,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;..."


In [24]:
df['tid'] = df['big_tid'].str.split(';', expand=True)[1]
df['full_orf'] = df.rel_start != 1
df['len'] = (df.rel_stop-df.rel_start).abs()+1

In [11]:
df.head()

,big_tid,frame,start,stop,rel_start,rel_stop,s_name,match_flag,s_align_percent,ident_percent,q_name,tid
0,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;NA",F1,423,3158,142,1052,ENSP00000286031.6,90_match,100,100,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;...","ENSG00000000460[3,10,3]"
1,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;NA",F1,423,3158,142,1052,ENSP00000352276.4,90_match,100,100,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;...","ENSG00000000460[3,10,3]"
2,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;NA",F1,423,3158,142,1052,ENSP00000490194.1,90_match,100,100,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;...","ENSG00000000460[3,10,3]"
3,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;NA",F1,423,3158,142,1052,ENSP00000490333.1,90_match,100,100,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;...","ENSG00000000460[3,10,3]"
4,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;NA",F1,423,3158,142,1052,ENSP00000389257.3,90_match,100,100,"ENSG00000000460.16;ENSG00000000460[3,10,3];NA;...","ENSG00000000460[3,10,3]"


In [12]:
df.match_flag.unique()

array(['90_match', 'no_hit', '50_match', 'bad_match'], dtype=object)

In [15]:
df.loc[(df.rel_start==1)&(df.tid=='ENSG00000049540[1,1,1]')]

,big_tid,frame,start,stop,rel_start,rel_stop,s_name,match_flag,s_align_percent,ident_percent,q_name,tid
3448819,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,2,2311,1,769,ENSP00000510104.1,90_match,100,100,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]"
3448820,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,2,2311,1,769,ENSP00000313565.6,90_match,100,100,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]"
3448821,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,2,2311,1,769,ENSP00000399499.1,90_match,100,100,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]"
3448822,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,2,2311,1,769,ENSP00000369950.5,90_match,100,100,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]"
3448823,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,2,2311,1,769,ENSP00000369926.4,90_match,100,100,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]"
3448824,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,2,2311,1,769,ENSP00000403961.1,90_match,100,100,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]"
3448825,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,2,2311,1,769,ENSP00000252034.7,90_match,100,100,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]"
3448826,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,2,2311,1,769,ENSP00000391129.1,90_match,100,100,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]"
3448827,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,2,2311,1,769,ENSP00000411092.1,90_match,100,100,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]"
3448828,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,2,2311,1,769,ENSP00000349540.5,90_match,100,100,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]"


In [19]:

df.loc[(df.rel_start==1)&(df.tid=='ENSG00000049540[1,1,1]')]

,big_tid,frame,start,stop,rel_start,rel_stop,s_name,match_flag,s_align_percent,ident_percent,q_name,tid,full_orf
3448819,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,2,2311,1,769,ENSP00000510104.1,90_match,100,100,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]",False
3448820,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,2,2311,1,769,ENSP00000313565.6,90_match,100,100,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]",False
3448821,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,2,2311,1,769,ENSP00000399499.1,90_match,100,100,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]",False
3448822,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,2,2311,1,769,ENSP00000369950.5,90_match,100,100,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]",False
3448823,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,2,2311,1,769,ENSP00000369926.4,90_match,100,100,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]",False
3448824,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,2,2311,1,769,ENSP00000403961.1,90_match,100,100,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]",False
3448825,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,2,2311,1,769,ENSP00000252034.7,90_match,100,100,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]",False
3448826,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,2,2311,1,769,ENSP00000391129.1,90_match,100,100,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]",False
3448827,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,2,2311,1,769,ENSP00000411092.1,90_match,100,100,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]",False
3448828,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,2,2311,1,769,ENSP00000349540.5,90_match,100,100,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]",False


In [25]:
df.loc[df.q_name=='ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA::chr7:74028050-74069957(+):F3:137:2311:46:769:724:M']

,big_tid,frame,start,stop,rel_start,rel_stop,s_name,match_flag,s_align_percent,ident_percent,q_name,tid,full_orf,len
3448842,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,137,2311,46,769,ENSP00000510104.1,90_match,100,100,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]",True,724
3448843,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,137,2311,46,769,ENSP00000313565.6,90_match,100,100,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]",True,724
3448844,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,137,2311,46,769,ENSP00000399499.1,90_match,100,100,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]",True,724
3448845,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,137,2311,46,769,ENSP00000369950.5,90_match,100,100,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]",True,724
3448846,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,137,2311,46,769,ENSP00000369926.4,90_match,100,100,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]",True,724
3448847,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,137,2311,46,769,ENSP00000403961.1,90_match,100,100,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]",True,724
3448848,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,137,2311,46,769,ENSP00000252034.7,90_match,100,100,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]",True,724
3448849,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,137,2311,46,769,ENSP00000391129.1,90_match,100,100,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]",True,724
3448850,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,137,2311,46,769,ENSP00000411092.1,90_match,100,100,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]",True,724
3448851,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,137,2311,46,769,ENSP00000349540.5,90_match,100,100,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]",True,724


In [23]:
769-46+1

724

In [22]:
len('MAGLTAAAPRPGVLLLLLSILHPSRPGGVPGAIPGGVPGGVFYPGAGLGALGGGALGPGGKPLKPVPGGLAGAGLGAGLGAFPAVTFPGALVPGGVADAAAAYKAAKAGAGLGGVPGVGGLGVSAGAVVPQPGAGVKPGKVPGVGLPGVYPGGVLPGARFPGVGVLPGVPTGAGVKPKAPGVGGAFAGIPGVGPFGGPQPGVPLGYPIKAPKLPGGYGLPYTTGKLPYGYGPGGVAGAAGKAGYPTGTGVGPQAAAAAAAKAAAKFGAGAAGVLPGVGGAGVPGVPGAIPGIGGIAGVGTPAAAAAAAAAAKAAKYGAAAGLVPGGPGFGPGVVGVPGAGVPGVGVPGAGIPVVPGAGIPGAAVPGVVSPEAAAKAAAKAAKYGARPGVGVGGIPTYGVGAGGFPGFGVGVGGIPGVAGVPGVGGVPGVGGVPGVGISPEAQAAAAAKAAKYGVGTPAAAAAKAAAKAAQFGLVPGVGVAPGVGVAPGVGVAPGVGLAPGVGVAPGVGVAPGVGVAPGIGPGGVAAAAKSAAKVAAKAQLRAAAGLGAGIPGLGVGVGVPGLGVGAGVPGLGVGAGVPGFGAVPGALAAAKAAKYGAAVPGVLGGLGALGGVGIPGGVVGAGPAAAAAAAKAAAKAAQFGLVGAAGLGGLGVGGLGVPGVGGLGGIPPAAAAKAAKYGAAGLGGVLGGAGQFPLGGVAARPGFGLSPIFPGGACLGKACGRKRK')

724

In [27]:
df = df.sort_values(by=['s_align_percent', 'full_orf', 'len'],
                    ascending=[False, False, False])

In [28]:
df.loc[df.tid=='ENSG00000049540[1,1,1]']

,big_tid,frame,start,stop,rel_start,rel_stop,s_name,match_flag,s_align_percent,ident_percent,q_name,tid,full_orf,len
3448858,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,137,2311,46,769,ENSP00000403162.1,90_match,116,86,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]",True,724
3448859,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,137,2311,46,769,ENSP00000369949.4,90_match,116,86,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]",True,724
3448860,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,137,2311,46,769,ENSP00000392575.1,90_match,116,86,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]",True,724
3448861,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,137,2311,46,769,ENSP00000480955.1,90_match,116,86,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]",True,724
3448835,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,2,2311,1,769,ENSP00000403162.1,90_match,116,86,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]",False,769
3448836,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,2,2311,1,769,ENSP00000369949.4,90_match,116,86,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]",False,769
3448837,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,2,2311,1,769,ENSP00000392575.1,90_match,116,86,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]",False,769
3448838,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,2,2311,1,769,ENSP00000480955.1,90_match,116,86,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]",False,769
3448863,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,137,2311,46,769,ENSP00000394549.1,90_match,108,85,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]",True,724
3448840,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;NA",F3,2,2311,1,769,ENSP00000394549.1,90_match,108,85,"ENSG00000049540.16;ENSG00000049540[1,1,1];NA;N...","ENSG00000049540[1,1,1]",False,769
